In [ ]:
%pip install -r requirements.txt

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.prompts import PromptTemplate,  ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM

from langchain.memory import ChatMessageHistory


In [ ]:

# Define the system message
llm = Bedrock(model_id='arn:aws:bedrock:us-west-2:637423382292:imported-model/7i06g1utels3', region_name='us-west-2', provider='meta')

system_message = '''You are an expert assistant that answers questions about different topics.

You are given some extracted parts from science papers along with a question.

If you don't know the answer, just say "I don't know." Don't try to make up an answer.

Use only the following pieces of context to answer the question at the end.

Do not use any prior knowledge.'''

from langchain.prompts import SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

template= '''<|system|>
{message}
<|end|>
'''

# A human message will contain the question and the context. The context will be automatically added by the retriever.
human_template = '''<|user|>
Context: {context}

Question is below:

Question: {question}

<|end|>
<|assistant|>
'''


assistant_template = '''<|assistant|>
{message}
<|end|>
'''

# Define the templates
SystemMessageTemplate = SystemMessagePromptTemplate.from_template(template)
HumanMessageTemplate = HumanMessagePromptTemplate.from_template(human_template)
AIMessageTemplate = AIMessagePromptTemplate.from_template(assistant_template)

system_msg = SystemMessageTemplate.format(message=system_message)


In [ ]:
from langchain.prompts import MessagesPlaceholder, PromptTemplate, ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    messages=[
    system_msg,
    human_template,
    ]
)

# As we can see our prompt is expecting three variables to be filled
print(chat_template)

rag_chain = chat_template | llm | StrOutputParser()

response = rag_chain.invoke({"question": 'What bands are available in HLS?', "context": ["The Harmonized Landsat and Sentinel-2 (HLS) project is an extension of research conducted at NASA's Goddard Space Flight Center in Greenbelt, MD, that takes input data from the joint NASA/USGS Landsat 8 and Landsat 9 and the ESA (European Space Agency) Sentinel-2A, Sentinel-2B, and Sentinel-2C satellites to generate a harmonized, analysis-ready surface reflectance data product with observations every two to three days.", "The HLS project is a major outcome of the Satellite Needs Working Group assessment in 2016. In that assessment, federal agencies and end users identified a need for more frequent Landsat-like observations to track short-term changes in vegetation and other land components to support agricultural monitoring and land cover classification at moderate to high resolution in both the visible and thermal components of the electromagnetic spectrum. Spectral similarities between the Landsat 8 Operational Land Imager (OLI), the Landsat 9 OLI-2, and the Sentinel-2 MultiSpectral Instrument (MSI) present an opportunity to harmonize data from these sensors to generate higher-frequency imagery products for land surface monitoring and applications.", "Previous versions of HLS data products produced by the HLS Science Team at Goddard had limited spatial coverage — only covering North America and other select global locations. The current version of the HLS algorithm is a cloud-based software stack that expands the spatial coverage to include all land masses globally, outside of Antarctica.", "HLS Harmonized Landsat and Sentinel-2 The Harmonized Landsat and Sentinel-2 (HLS) project is an extension of research conducted at NASA's Goddard Space Flight Center in Greenbelt, MD, that takes input data from the joint NASA/USGS Landsat 8 and Landsat 9 and the ESA (European Space Agency) Sentinel-2A, Sentinel-2B, and Sentinel-2C satellites to generate a harmonized, analysis-ready surface reflectance data product with observations every two to three days.The HLS project is a major outcome of the Satellite Needs Working Group assessment in 2016. In that assessment, federal agencies and end users identified a need for more frequent Landsat-like observations to track short-term changes in vegetation and other land components to support agricultural monitoring and land cover classification at moderate to high resolution in both the visible and thermal components of the electromagnetic spectrum. Spectral similarities between the Landsat 8 Operational Land Imager (OLI), the Landsat 9 OLI-2, and the Sentinel-2 MultiSpectral Instrument (MSI) present an opportunity to harmonize data from these sensors to generate higher-frequency imagery products for land surface monitoring and applications.Previous versions of HLS data products produced by the HLS Science Team at Goddard had limited spatial coverage — only covering North America and other select global locations. The current version of the HLS algorithm is a cloud-based software stack that expands the spatial coverage to include all land masses globally, outside of Antarctica.Two data products are generated as part of the HLS project: the L30 data product generated with Landsat 8 and Landsat 9 data, and the S30 product generated using Sentinel-2 data. These data are available through Earthdata Search as well as through NASA's Land Processes Distributed Active Archive Center (LP DAAC). Feedback or questions about HLS data products can be made in the Earthdata Forum for HLS."]})

print(response)